## This is where I will do the classifications for my data

* First, I need to load the data and quickly merge the dataframes...

In [74]:
import pandas as pd
import numpy as np 
import utils 
import importlib
importlib.reload(utils)


# First we need to import our data from a csv file: 
df = pd.read_csv("AutoSleep.csv", header=0)
days_df = pd.read_csv("days_of_week.csv", header=0)

df_2 = pd.read_csv("AutoSleep_part2.csv", header=0)
days_part_2 = pd.read_csv("days_test.csv", header=0)
join = [df, df_2]
join_days = [days_df, days_part_2]

df = pd.concat(join)
df = df.iloc[:-1 , :]
# df.drop(df.index[-1], axis=0, inplace=True)
days_df = pd.concat(join_days)
days_df = days_df.iloc[:-1 , :]
# days_df.drop(days_df.index[-1], axis=0, inplace=True)

for col in df:
    check_for_nan = df[col].isnull().values.any()
    if check_for_nan == True:
        # print (col, check_for_nan, df[col].isnull().count())
        df.drop(col, axis=1, inplace=True)

df.drop(['ISO8601', 'toDate', 'inBed',
       'awake', 'fellAsleepIn', 'sessions', 'asleepAvg7', 'efficiencyAvg7', 
       'qualityAvg7', 'sleepBPMAvg7', 'wakingBPMAvg7','hrvAvg7'], axis=1, inplace=True)

merged_df = days_df.merge(df, on='fromDate')
merged_df = utils.clean_sleep(df, 'asleep')

### KNN Classifier 

In [75]:
# distribution of the data 
attribute = merged_df['efficiency']
attribute.describe()

count     60.000000
mean      94.180000
std       10.214344
min       55.300000
25%       94.550000
50%       98.750000
75%      100.000000
max      100.000000
Name: efficiency, dtype: float64

In [76]:
# Apply scaling 
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler

le = preprocessing.LabelEncoder()
df_encoded = merged_df.copy()

clean_merged = df_encoded.drop(['fromDate', 'efficiency'], axis=1)
for col in clean_merged:
    # print(col)
    df_encoded[col] = le.fit_transform(clean_merged[col])
    # list(le.classes_,)

# X_train = df_encoded.drop(['fromDate','efficiency'], axis=1)
# y_train = df_encoded['efficiency']
# y_train = y_train.astype(float)
# # print(X_train, y_train, sep="\n")
# # print(test_df_encoded)

X = df_encoded.drop(['fromDate', 'efficiency'], axis=1)
y_ser = df_encoded['efficiency']
y = y_ser.to_frame()

scaler = MinMaxScaler()
X = scaler.fit_transform(X) 
# # typically fit and transform are combined fit_transform()
# # print("******\nX-train scaled:", X_train_scaled, "\n*******")
# X_test = test_df_encoded.drop(['fromDate','efficiency'], axis=1)
# X_test_scaled = scaler.transform(X_test)
# # print(X_test_scaled)

# # print(y_train)
# # print(type(y_train))

In [77]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold 

# print(len(X_test_scaled), len(y_train), len(X_train_scaled))
# print(type(y_train))

# # train/build/fit
# knn_clf = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
# knn_clf.fit(X_train_scaled, y_train)

# # predict
# y_predicted = knn_clf.predict(X_test_scaled)
# print("the predicted values are:", y_predicted)

# a = merged_test['efficiency'].to_list()
# y_actual = [int(a) for a in a]
# print(y_actual)

# for i in range(len(y_predicted)):
#     print(y_actual[i], y_predicted[i])

In [90]:
print(len(X)*0.25) # ceil(4.5) -> 5
# random_state is used for reproducible
# shuffling
# stratify is used to get a similar distribution
# of class labels in the train/test sets
# as the original dataset
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.25, n_splits=2,
    random_state=0, stratify=y)
# print(X_test)
# print(y_test)

knn_clf = KNeighborsClassifier(n_neighbors=3,
    metric="euclidean")
knn_clf.fit(X_train, y_train)
y_predicted = knn_clf.predict(X_test)
print("y predicted:", y_predicted)
print("y actual (AKA y true)", y_test.tolist())
# 80% accuracy
accuracy = knn_clf.score(X_test, y_test)
print("accuracy:", accuracy)
accuracy = accuracy_score(y_test, y_predicted)
print("accuracy:", accuracy)


15.0


TypeError: train_test_split() got an unexpected keyword argument 'n_splits'